# Imports

In [1]:
import numpy as np
import pandas as pd
import random
import math
import time

# Loading the Data and Discarding Some

In [2]:
data = pd.read_csv('./clean_data.csv',sep='\t',encoding='utf-8',index_col=0)

We will drop all the unnecessary columns of the DataFrame. The ones used for prediction will be the statistics related to the current form of the team in the league. These are the ones that will be available in real-life for prediction. Of course the predictor column, 'watch' is kept.

In [3]:
data.drop(labels=['Date','HomeTeam','AwayTeam',
                  'FTHG','FTAG','HS','AS','HST',
                  'AST','HTHG','HTAG','HF','AF',
                  'FTR','HTR','HY','AY','HR',
                  'AR','Season','WHH','WHD','WHA'],
         inplace=True,
         axis=1)
# Redorder the columns so we have home statistics, then away statistics
# then the classifier.
data = data[['HGS','HGA','HYC','HRC','HWW','AGS','AGA','AYC','ARC','AWW','watch']]

In [4]:
data.sample(10)

,HGS,HGA,HYC,HRC,HWW,AGS,AGA,AYC,ARC,AWW,watch
3489,1.176471,1.058824,1.588235,0.176471,0.352941,1.764706,1.411765,1.882353,0.235294,0.470588,0
819,1.500000,1.333333,1.833333,0.083333,0.333333,1.000000,1.000000,1.153846,0.230769,0.153846,0
43,0.800000,1.800000,2.400000,0.000000,0.200000,2.200000,2.400000,2.600000,0.000000,1.000000,0
3215,1.000000,1.653846,1.576923,0.076923,0.346154,1.160000,1.840000,2.040000,0.080000,0.520000,1
3361,2.000000,1.666667,2.666667,0.000000,0.666667,0.666667,1.333333,0.666667,0.333333,0.000000,0
4479,1.428571,1.285714,1.428571,0.000000,0.571429,0.857143,1.000000,1.428571,0.142857,0.142857,0
4672,2.538462,1.230769,1.423077,0.000000,0.653846,1.269231,1.384615,1.692308,0.038462,0.230769,1
4300,1.680000,1.160000,1.320000,0.080000,0.400000,1.320000,1.760000,1.960000,0.080000,0.360000,1
1774,1.000000,1.500000,1.562500,0.062500,0.312500,1.812500,0.875000,1.687500,0.000000,0.375000,0
1684,0.714286,1.428571,1.714286,0.000000,0.285714,1.166667,0.500000,1.500000,0.000000,0.166667,1


We are trying to classify a new datapoint based on the previous game statistics in the league and betting odds for the coming game, into worth watching or not watching. The model will be trained on the data from previous seasons. This is a situation where k-Nearest Neighbors is appropriate. The model will take a new data point, and classify it based on the k-nearest datapoints. If the k-nearest have a majority of them as worth watching, them the new one will be classified as also worth watching. This model is going to treat the data as points living in a 13-dimensional space and you can imagine points being colored red if they are worth watching and blue if not.

# Implementing the Predictor by Hand

Rather than using an implementation of k-Nearest Neighbors from a package like scikit-learn I will implement it using just numpy and pandas.

In [5]:
# This function normalizes all the columns of a dataframe
# to have mean 0 and standard deviation 1. This ensures 
# that no column has a dominant impact on the model
def normalize(df):
    return (df - df.mean()) / df.std()

In [6]:
# This function takes the dataframe and a newpoint to classify. It returns
# a Series object with the distance from each of the rows of the dataframe
# to the newpoint. We use Euclidean distance
def find_distances(df, newpoint):
    assert type(newpoint) == pd.Series, 'The new point must be a pd.Series object'
    assert newpoint.shape[0] == (df.shape[1] - 1), f'The point is the wrong shape'
    distances = []
    for i, row in df.drop('watch', axis=1).iterrows():
        distances.append(np.linalg.norm(row - newpoint))
    return pd.Series(distances)        

In [7]:
# Given a dataframe, a series of distances (found above)
# and a value of k this will return the
# k nearest rows of the dataframe
def nearest(df, distances, k):
    assert type(k) == int, 'k must be an integer'
    assert 1 <= k <= df.shape[0], 'k must be a positive integer'
    assert type(distances) == pd.Series, 'Distances must be a pd.Series object'
    copy = df.copy()
    copy['distances'] = distances
    return copy.sort_values(by='distances',axis=0)[:k].drop('distances', axis=1)

In [8]:
# Given a dataframe with k rows will return the majority
# ruling on 'watch' variable, in case of a tie it will return
# 0 = 'not worth watching'
def worth_watching(nearest):
    average = nearest['watch'].sum() / nearest['watch'].count()
    return int(average > 0.5)

In [9]:
# Now we can put these inside a general predcition funciton
def predict(dataframe, newpoint, k):
    df = normalize(dataframe)
    # We must apply the same normalizing function to the new point
    normed_point = (newpoint - dataframe.mean()) / dataframe.std()
    # Drop the column it has picked up from the normalization
    normed_point.drop('watch', inplace=True)
    dists = find_distances(df, normed_point)
    near = nearest(df, dists, k)
    return worth_watching(near)

In [10]:
# Testing
newpoint = pd.Series({'HGS':3,'HGA':0,'HYC':0,
                      'HRC':0,'HWW':1,'AGS':3.5,
                      'AGA':2.7,'AYC':2.5,'ARC':0,
                      'AWW':1})
for k in range(1, 20):
    print(f'k={k}, watch={predict(data, newpoint, k)}')

k=1, watch=1
k=2, watch=0
k=3, watch=1
k=4, watch=0
k=5, watch=0
k=6, watch=0
k=7, watch=0
k=8, watch=0
k=9, watch=0
k=10, watch=0
k=11, watch=0
k=12, watch=0
k=13, watch=0
k=14, watch=0
k=15, watch=0
k=16, watch=0
k=17, watch=0
k=18, watch=0
k=19, watch=0


# Model Selection and Testing

To determine the best choice of k-value for the model we will be splitting the data into training and testing subsets. I will assign 80% to training and 20% to testing. These will be randomly assigned. I will run 10-fold cross validation which means that the training subset will be split into 10 same size non-intersecting subsets. We will fit the model for values of k on 90% of the training data then validate it on the remaining 10%. Once we have done this 10 times we will have an accuracy for that value of k. Once we have done this for all values of k the one with the highest accuracy from 10-fold cross-validation will be selected as the optimal value.

Once we have tested all the values of k we will assess the final accuracy of the model by fitting on all the training data and testing on the test data. This process eliminates bias in the final accuracy value.

## Subsetting

You will notice below that the training fraction is only 10% of the data! This is because the prediction takes a very long time to run. This is due to the fact that when we are classifying a new data point we must calculate distance from this new point to all of the training points. In this kind of implementation there are no ways around it. I am going to use 10% for my own implementation and then we will use a package below for the full 80% training. It will be interesting to see the difference

In [11]:
random.seed(2718) # Euler
training_frac = 0.1 # Train on over 600 games
testing_frac = 0.05 # Test on over 300 games
indices_train = random.sample([i for i in range(len(data))], 
                              math.floor(training_frac * len(data)))
indices_test = random.sample([i for i in range(len(data)) if i not in indices_train], 
                             math.floor(testing_frac * len(data)))
data_train = data.iloc[indices_train].reset_index()
data_test = data.iloc[indices_test].reset_index()

## Finding the optimal value of k

In [12]:
accuracy = []
for k in range(1, 20):
    start = time.time()
    correct = 0
    for i in range(10):
        validate_indices_low = math.floor(i / 10 * len(data_train))
        validate_indices_high = math.floor((i + 1) / 10 * len(data_train))
        data_validate = data_train.iloc[range(validate_indices_low, 
                                              validate_indices_high)]
        data_fit = data_train.iloc[[i for i in range(len(data_train)) 
                                    if i not in range(validate_indices_low, 
                                                      validate_indices_high)]]
        watch_pred = pd.DataFrame(data_validate['watch'])
        watch_pred['pred'] = data_validate.apply(lambda row: predict(data_fit, row.drop('watch'), k), axis=1)
        correct += len(watch_pred[watch_pred['watch'] == watch_pred['pred']])
        print(f'{i} fold done for k={k}', end='. ')
    accuracy.append(correct / len(data_train))
    end = time.time()
    print(f'\nThe validation accuracy for k={k} is {round(accuracy[k - 1] * 100, 3)}% and took {end - start} seconds to run')

0 fold done for k=1. 1 fold done for k=1. 2 fold done for k=1. 3 fold done for k=1. 4 fold done for k=1. 5 fold done for k=1. 6 fold done for k=1. 7 fold done for k=1. 8 fold done for k=1. 9 fold done for k=1. 
The validation accuracy for k=1 is 60.096% and took 191.13221383094788 seconds to run
0 fold done for k=2. 1 fold done for k=2. 2 fold done for k=2. 3 fold done for k=2. 4 fold done for k=2. 5 fold done for k=2. 6 fold done for k=2. 7 fold done for k=2. 8 fold done for k=2. 9 fold done for k=2. 
The validation accuracy for k=2 is 64.263% and took 189.2847671508789 seconds to run
0 fold done for k=3. 1 fold done for k=3. 2 fold done for k=3. 3 fold done for k=3. 4 fold done for k=3. 5 fold done for k=3. 6 fold done for k=3. 7 fold done for k=3. 8 fold done for k=3. 9 fold done for k=3. 
The validation accuracy for k=3 is 60.737% and took 185.27041006088257 seconds to run
0 fold done for k=4. 1 fold done for k=4. 2 fold done for k=4. 3 fold done for k=4. 4 fold done for k=4. 5 fol

## Model Testing

As can be seen from the output above, the optimal value of $k$ (having only checked 1 to 19) on the training/validation data was $k = 10$. We will finally test the model on the reserved testing data to get a true final accuracy. The values in the confusion matrix have also been presented.

In [13]:
final_k = 10
watch_pred = pd.DataFrame(data_test['watch'])
watch_pred['pred'] = data_test.apply(lambda row: predict(data_train, row.drop('watch'), final_k), axis=1)
correct = len(watch_pred[watch_pred['watch'] == watch_pred['pred']])
accuracy = correct / len(data_test)

In [14]:
print(f'Final accuracy of the model on testing data is: {accuracy}')
true_positives = watch_pred.loc[(watch_pred['watch'] == 1) & (watch_pred['pred'] == 1)]
true_negatives = watch_pred.loc[(watch_pred['watch'] == 0) & (watch_pred['pred'] == 0)]
false_positives = watch_pred.loc[(watch_pred['watch'] == 0) & (watch_pred['pred'] == 1)]
false_negatives = watch_pred.loc[(watch_pred['watch'] == 1) & (watch_pred['pred'] == 0)]
print(f'True positives: {len(true_positives)}')
print(f'True negatives: {len(true_negatives)}')
print(f'False positives: {len(false_positives)}')
print(f'False negatives: {len(false_negatives)}')
print(f'Total number of games: {len(data_test)}')

Final accuracy of the model on testing data is: 0.6538461538461539
True positives: 11
True negatives: 193
False positives: 12
False negatives: 96
Total number of games: 312


So we end with a final accuracy of 65.4% which is not bad at all, considering we are predicting sporting events which have a lot of randomness to their results. But remember, we only used 10% of the data for training and validation and then 5% for testing, so we can't really claim that the k-Nearest Neighbors will work this well in practice but the results are encouraging.

On the other hand, looking at the values from the confusion matrix is little more concerning. There are only 11 true positives out of the 312 games tested. This means that over roughly 30 weekends of football, at most 11 will have a game that is predicted to be worth watching and actually be worth watching.

Even more of a problem is the number of false positives. These would be games the model tells us to watch but turn out to be dull.

# Predicting Using Scikit-Learn

We wil be able to use the full 80% of the training/validation data with the functions of Scikit-Learn as they are made by people who know a lot more about code optimisation that myself.

In [15]:
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [16]:
random.seed(2718) # Euler
training_frac = 0.8
testing_frac = 0.2
indices_train = random.sample([i for i in range(len(data))], 
                              math.floor(training_frac * len(data)))
indices_test = random.sample([i for i in range(len(data)) if i not in indices_train], 
                             math.floor(testing_frac * len(data)))
data_train = data.iloc[indices_train].reset_index()
data_test = data.iloc[indices_test].reset_index()

In [17]:
accuracy = []
best_accuracy = 0
start = time.time()
for k in range(1, 100):
    # By passing distance we give closer points a higher weight
    # I have set the algorithm to brute meaning that the distance to 
    # all points will be checked. As I have above
    model = KNeighborsClassifier(k, weights='distance',algorithm='brute')
    correct = 0
    for i in range(10):
        validate_indices_low = math.floor(i / 10 * len(data_train))
        validate_indices_high = math.floor((i + 1) / 10 * len(data_train))
        data_validate = data_train.iloc[range(validate_indices_low, 
                                              validate_indices_high)]
        data_fit = data_train.iloc[[i for i in range(len(data_train)) 
                                    if i not in range(validate_indices_low, 
                                                      validate_indices_high)]]
        watch_pred = pd.DataFrame(data_validate['watch'])
        model.fit(data_fit.drop('watch', axis=1), data_fit['watch'])
        watch_pred['pred'] = model.predict(data_validate.drop('watch', axis=1))
        correct += len(watch_pred[watch_pred['watch'] == watch_pred['pred']])
    if correct / len(data_train) > best_accuracy:
        best_accuracy = correct / len(data_train)
        best_k = k
end = time.time()
print(f'This took {round(end - start, 0)} seconds to run')
print(f'The best validation accuracy was for k={best_k} and is {best_accuracy}')

This took 91.0 seconds to run
The best validation accuracy was for k=97 and is 0.649119295436349


In [18]:
final_k = 97
# Same parameters as above
model = KNeighborsClassifier(final_k, weights='distance', algorithm='brute')
watch_pred = pd.DataFrame(data_test['watch'])
# This time we fit on all the training data
model.fit(data_train.drop('watch', axis=1), data_train['watch'])
# Predict on the remaining, unseen 20% of testing data
watch_pred['pred'] = model.predict(data_test.drop('watch', axis=1))
correct = len(watch_pred[watch_pred['watch'] == watch_pred['pred']])
accuracy = correct / len(data_test)
print(f'The accuracy on the testing data for k={final_k} is {round(accuracy * 100, 3)}')
print(confusion_matrix(y_true = watch_pred['watch'], y_pred=watch_pred['pred']))
print(f'Total number of games {len(data_test)}')

The accuracy on the testing data for k=97 is 66.133
[[794  47]
 [376  32]]
Total number of games 1249


The results above show that the k-Nearest Neighbors classifier is performing just as well as my implementation but has selected a much higher value of k as optimal. To run my implmentation of k-nearest neighbors for $k=97$ would take a very long time. Again, we get dissapointing results from the confusion matrix. The number of true positives is only 32 out of a total of 1249 games! There are many false negatives which means missed opportunities to watch exciting games and many false positives.

**So despite a good final accuracy for the model, it may not work so well in practice. Can we do better?**